## Init

In [51]:
%load_ext autoreload
%autoreload 2
%aimport

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Modules to reload:
all-except-skipped

Modules to skip:



In [52]:
import numpy as np
import polars as pl
import pandas as pd
import pyarrow as pa
import matplotlib.pyplot as plt

from abc import ABCMeta, abstractmethod
from dataclasses import dataclass
from sklearn.model_selection import train_test_split
import pandas as pd
import pyarrow as pa
from typing import Callable
import re
from datetime import datetime
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier
import xgboost as xgb

from catboost import CatBoostClassifier

from collections import Counter

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.metrics import RocCurveDisplay

from utils.data import clean_test_to_fit_answer_sheet, submit_answer

pl.Config.set_tbl_rows(999)
pl.Config.set_tbl_cols(999)
pl.Config.set_tbl_width_chars(9999)
pl.Config.set_fmt_str_lengths(9999)

polars.config.Config

In [140]:
DATAPATH = './data'

answer_sheet = pl.read_csv(f'{DATAPATH}/answer_sheet.csv').select(pl.col('name').alias('Name'), pl.col('age').alias("Age"), pl.col('survived').alias("Survived"))

train = pl.read_csv(f'{DATAPATH}/train.csv')
test = clean_test_to_fit_answer_sheet(pl.read_csv(f'{DATAPATH}/test.csv')).join(answer_sheet, on=['Name', 'Age'], how='inner')
submission_sample = pl.read_csv(f'{DATAPATH}/gender_submission.csv')

In [141]:
pl.Config.set_tbl_formatting("UTF8_FULL_CONDENSED")
pl.Config.set_tbl_width_chars(175)
pl.Config.set_tbl_cols(99)

pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Data Exploration

In [ ]:
train_explore = train.clone()

In [ ]:
# train_explore.head()
train_explore.describe()

In [ ]:
'''
Name
Ticket
Cabin
'''
(
    train_explore
    .select(
        '*',
        pl.col('Name').str.split(by=', ').apply(lambda x: x[0]).alias('LastName'),
        pl.col('Name').str.split(by=', ').apply(lambda x: x[1]).str.split(by=' ').apply(lambda x: x[0]).alias('Honorific')
    )
)

# train_explore.select('Name', pl.col('Ticket')).sort('Ticket').head(5)

# train_explore.select('Cabin', pl.col('Ticket')).sort('Cabin').head(5)
# train_explore.groupby('Cabin').agg(pl.n_unique('PassengerId').alias('counts')).sort('counts', descending=True)
# train_explore.filter(pl.col('Cabin')=='C23 C25 C27')
# display(train_explore.groupby('Ticket').agg(pl.n_unique('Cabin').alias('cabincount')).filter(pl.col('cabincount') > 1).head(5))
# train_explore.filter(pl.col('Ticket') == '17421')

In [ ]:
def get_features_to_outcome_effect(df: pl.DataFrame, group_cols: list[str], outcome_col: str) -> None:
    for col in group_cols:
        if df[col].dtype in [pl.Float32, pl.Float64]:
            df = (
                df
                .with_columns(pl.col(col).qcut(5).alias(col))
            )
    return df.groupby(group_cols).agg(pl.mean(outcome_col)).sort(group_cols, descending=False)

# single_variable_relation_to_outcome(train_explore, ['Sex'], 'Survived')
# single_variable_relation_to_outcome(train_explore, ['Pclass'], 'Survived')
# single_variable_relation_to_outcome(train_explore, ['Age'], 'Survived')
# single_variable_relation_to_outcome(train_explore, ['SibSp'], 'Survived')
# single_variable_relation_to_outcome(train_explore, ['Parch'], 'Survived')
# single_variable_relation_to_outcome(train_explore, ['Fare'], 'Survived')
# single_variable_relation_to_outcome(train_explore, ['Embarked'], 'Survived')

## Transform data

In [307]:
from utils.baseclass import Dataset
from utils.data import TitanicDataset
from utils.decorators import PolarsCompatibleTransformer
from utils.transformers import TransformColToCategorical, TransformStringColToNumeric, TransformColToBins, DropColumns, FillNull, AddRandomColumn, PipelineCompatibleCatBoostClassifier

from utils.transformers.Name import MakeNameFeatures, CleanHonorific, CleanLastName
from utils.transformers.Fare import CleanFare
from utils.transformers.Embarked import CleanEmbarked
from utils.transformers.Age import CleanAge
from utils.transformers.Cabin import CleanCabin
from utils.transformers.FamilyIndicator import MakeIsWomanOrBoyIndicator, MakeFamilySurvivedRate, AddFamilySize, AddIsMarried

In [342]:
PIPELINE = Pipeline([
    ('make_name_features', MakeNameFeatures()),
    ('clean_honorific', CleanHonorific()),
    ('clean_lastname', CleanLastName()),
    ('transform_honorific', TransformColToCategorical('Honorific', replace_original=False)),
    ('transform_honorificgrouped', TransformColToCategorical('HonorificGrouped', replace_original=False)),
    
    ('add_family_size', AddFamilySize()),
    ('add_is_married', AddIsMarried()),
    ('transform_familysize_group', TransformColToCategorical('FamilySizeGroup')),
    ('add_womanboy_indicator', MakeIsWomanOrBoyIndicator()),
    ('make_lastname_surviverate', MakeFamilySurvivedRate(['LastName'])),
    ('make_ticket_surviverate', MakeFamilySurvivedRate(['Ticket'])),
    ('make_cabin_surviverate', MakeFamilySurvivedRate(['Cabin'])),
    # ('make_lastnameclean_surviverate', MakeFamilySurvivedRate(['LastNameClean'])),

    ('transform_sex', TransformColToCategorical('Sex', replace_original=False)),

    ('clean_embarked', CleanEmbarked()),
    ('transform_embarked', TransformColToCategorical('Embarked', replace_original=False)),
    
    ('clean_fare', CleanFare()),
    ('transform_fare', TransformColToBins('Fare', replace_original=False, return_numeric=True)),
    
    ('clean_age', CleanAge()),
    ('transform_age', TransformColToBins('Age', replace_original=False, return_numeric=True)),
    
    ('clean_cabin', CleanCabin()),
    ('transform_cabin_firstletter', TransformColToCategorical('CabinFirstLetter', replace_original=False)),
    ('transform_cabin_firstlettergroup', TransformColToCategorical('CabinFirstLetterGroups', replace_original=False)),
    
    ('drop_unneeded_cols', DropColumns(
        cols_to_drop=['PassengerId', 'Survived'], # + ['Pclass', 'Age', 'Parch', 'SibSp', 'Fare'],
        regex_to_drop='groupid_*',
        drop_strings = True
    )),

    ('fill_nulls', FillNull(-1)),
    
    # ('xgbc', XGBClassifier(enable_categorical=True, tree_method='approx'))
    # ('xgbc', XGBClassifier(random_state=123))
    ('catboost', PipelineCompatibleCatBoostClassifier(random_seed=123, max_depth=10, subsample=0.5, colsample_bylevel=0.5, n_estimators=3000))
])
train_transformed = TitanicDataset(train.clone(), 'Survived', retain_outcome_col=True, test_size=0.2)
test_transformed = TitanicDataset(test.clone(), 'Survived', predict_only=True)

# df_clean = PIPELINE.fit_transform(X=train_transformed.xtrain, y=train_transformed.ytrain)
# df_clean.head(3)
# df_clean.describe(include='all')

PIPELINE.fit(X=train_transformed.xtrain, y=train_transformed.ytrain);
print(accuracy_score(train_transformed.ytrain, PIPELINE.predict(train_transformed.xtrain)))
print(accuracy_score(train_transformed.ytest, PIPELINE.predict(train_transformed.xtest)))
print(accuracy_score(test_transformed.y, PIPELINE.predict(test_transformed.X)))

2023-09-04 23:40:09.919 | WARNING  | utils.data:__post_init__:28 - Retaining outcome column `Survived` in X. You **must** drop this before training


The predicate '[(col("IsWomanOrBoy")) == (1)]' in 'when->then->otherwise' is not a valid aggregation and might produce a different number of rows than the groupby operation would. This behavior is experimental and may be subject to change
The predicate '[(col("IsWomanOrBoy")) == (1)]' in 'when->then->otherwise' is not a valid aggregation and might produce a different number of rows than the groupby operation would. This behavior is experimental and may be subject to change
The predicate '[(col("IsWomanOrBoy")) == (1)]' in 'when->then->otherwise' is not a valid aggregation and might produce a different number of rows than the groupby operation would. This behavior is experimental and may be subject to change
/Users/yongjian.tek/.pyenv/versions/3.10.4/envs/kaggle/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(es

0.9199438202247191
0.8379888268156425
0.7751196172248804


In [304]:
test_preds = PIPELINE.transform(test_transformed.X)

test_preds['preds'] = np.select(
    [(test_preds['HonorificGrouped_categorical'] == 2) & (test_preds['FamilyWomanOrBoySurvivedRate_LastName'] == 0),
    (test_preds['HonorificGrouped_categorical'] == 0) & (test_preds['FamilyWomanOrBoySurvivedRate_LastName'] == 1),
    (test_preds['HonorificGrouped_categorical'] == 2)],
    [0,1,1], default=0
)
accuracy_score(test_transformed.y, test_preds['preds'])

0.6220095693779905

In [279]:
# test_transformed['preds'] = PIPELINE.predict(test_transformed.X)
# test_transformed['preds'] = np.where(test_transformed['Title'] == 'male')

In [32]:
# Counter(df_clean['Honorific'])

In [273]:
# RocCurveDisplay.from_estimator(PIPELINE, train_transformed.xtest, train_transformed.ytest)
# print(accuracy_score(train_transformed.ytrain, PIPELINE.predict(train_transformed.xtrain)))
# print(accuracy_score(train_transformed.ytest, PIPELINE.predict(train_transformed.xtest)))
# print(accuracy_score(test_transformed.y, PIPELINE.predict(test_transformed.X)))

In [81]:
manual_preds = (
    pl.from_pandas(test_transformed.X)
    .with_columns(
        pl.Series(PIPELINE.predict(test_transformed.X)).alias('model_preds')
    )
    .with_columns(
        pl.col('Name').str.split(", ").apply(lambda x: x[0]).alias('LastName'),
        pl.col('Name').str.split(", ").apply(lambda x: x[1]).str.split('. ').apply(lambda x: x[0]).alias('Honorific')
    )
    .with_columns(
        pl.when(
            ((pl.col('Sex') == 'female') & (pl.col("Age") <= 18)) | 
            (pl.col('Honorific') == 'Master')
        ).then(
            pl.lit(1)
        ).otherwise(
            # pl.col('model_preds')
            pl.lit(0)
        ).alias('Survived')
    )
)
# manual_preds.head()
# manual_preds.sort('Honorific', 'Sex').head(50)

In [82]:
from datetime import datetime
import os

PassengerId,Survived
i64,i32
892,0
893,0
894,0
895,0
896,0


100%|██████████| 2.77k/2.77k [00:03<00:00, 777B/s]


Successfully submitted to Titanic - Machine Learning from Disaster

0

## TODO

In [ ]:
# Basic prediction: Predict men die, women live
# Basic prediction: Women live. Men who are "master" live. All else die

# Remove correlated features?
# Model tuning
# Stacking models
# Ensembling models